# Brain Tumor Detection

In [ ]:
import os
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm
import io
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Activation,Concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [ ]:
# Confirmer les chemins des fichiers
base_path = os.path.join(os.getcwd(), "Brain-Tumor-Classification")
for dirname, _, filenames in os.walk(base_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Récupérer les classes
classes = os.listdir(os.path.join(base_path, "Training"))
print(classes)

In [ ]:
X = [] # images 
y = [] # classes
image_size = 150 # taille des images

In [ ]:
from PIL import Image

# Boucle sur les dossiers Training et Testing
for dataset_type in ["Training", "Testing"]:
    for class_name in classes: # Boucle sur les classes
        folder_path = os.path.join(base_path, dataset_type, class_name) # Chemin du dossier de la classe actuelle

        # Boucle sur les fichiers du dossier actuel
        for image_file in tqdm(os.listdir(folder_path)):
            img_path = os.path.join(folder_path, image_file) # Chemin de l'image actuelle
            try:
                img = Image.open(img_path) # Ouvrir l'image
                img = img.resize((image_size, image_size)) # Redimensionner l'image
                img_array = np.array(img) # Convertir l'image en tableau numpy
                if len(img_array.shape) != 3 or img_array.shape[2] != 3: # Vérifier que l'image est en couleur (3 canaux)
                    print(f"Image sans les 3 canaux pour {img_path}")
                    continue
                X.append(img_array) # Ajouter l'image au tableau X
                y.append(class_name) # Ajouter la classe au tableau y
            except Exception as e:
                print(f"Impossible d'ouvrir l'image {img_path}: {e}")

In [ ]:
X = np.array(X) # convertir la liste des images en tableau
y = np.array(y) # convertir la liste des classes en tableau

In [ ]:
print(f"Nombre d'images chargées: {len(X)}") 
print(f"Nombre de label chargées: {len(y)}")
print(X.shape, y.shape) # Afficher la taille d'échantillon de X et y, ainsi que la taille d'une image et ses canaux

In [ ]:
k = 0 # index pour les images
figure, ax = plt.subplots(1,4,figsize=(20,20)) # 1 ligne, 4 colonnes
figure.text(s="Echantillon d'images pour chaque classe", size=20,fontweight='bold',y=0.62,x=0.5,ha='center',va='center')
for i in classes:
    j=0 # index pour les classes
    while True :
        if y[j]==i: # si la classe y[j] est égale à i
            ax[k].imshow(X[j]) # afficher l'image correspondante à la X[j]
            ax[k].set_title(y[j]) # afficher la classe de l'image en titre
            ax[k].axis('off') # ne pas afficher les axes
            k+=1
            break
        j+=1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=42) # 80% pour le training et 20% pour le test

In [ ]:
y_train_new = [] # liste pour les classes du training
for i in y_train: 
    y_train_new.append(classes.index(i)) # remplacer les noms des classes par leurs indices
y_train = y_train_new # mettre à jour les classes du training
y_train = tf.keras.utils.to_categorical(y_train) # convertir les classes en one-hot encoding

y_test_new = [] # liste pour les classes du test
for i in y_test: 
    y_test_new.append(classes.index(i)) # remplacer les noms des classes par leurs indices
y_test = y_test_new # mettre à jour les classes du test
y_test = tf.keras.utils.to_categorical(y_test) # convertir les classes en one-hot encoding

In [ ]:
img_datagen = ImageDataGenerator( # faire de la data augmentation
    rotation_range=30, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    zoom_range=0.2, 
    horizontal_flip=True) 

img_datagen.fit(X_train) # Fit la data augmentation
img_datagen.fit(X_test) # fit la data augmentation


In [ ]:
# Créer le modèle
model = Sequential() # modèle séquentiel

# Ajouter les couches de convolution, activation, normalisation et pooling
model.add(Conv2D(64, (3, 3), padding='same',input_shape=(image_size,image_size,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization()) 

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())
model.add(Dropout(0.35)) #64 --> 42

model.add(Conv2D(64, (3, 3), padding='same')) 
model.add(Activation('relu'))
model.add(BatchNormalization())

# Ajouter les couches de Flatten, Dropout, Dense, Activation et BatchNormalization
model.add(Flatten()) 
model.add(Dropout(0.5)) 
model.add(Dense(512)) 
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(4)) 
model.add(Activation('softmax'))

model.summary()

In [ ]:
# Compiler le modèle
model.compile(optimizer = 'adam',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

In [ ]:
# Entraîner le modèle
history = model.fit(X_train,y_train,validation_split=0.1, epochs=10, verbose=1, batch_size=32)

In [ ]:
# Evaluer le modèle
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'])
plt.show()

In [ ]:
# Evaluer le modèle
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'])
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

# Tester le modèle sur le training set
predictions_tr = model.predict(X_train)
predictions_tr = [np.argmax(x) for x in predictions_tr]

accuracy_tr = accuracy_score(np.argmax(y_train, axis=1), predictions_tr)
print('Training accuracy = %.4f' % accuracy_tr)

f1_tr = f1_score(np.argmax(y_train, axis=1), predictions_tr, average='weighted')
print('Training F1 score = %.4f' % f1_tr)

confusion_mtx_tr = confusion_matrix(np.argmax(y_train, axis=1), predictions_tr)

# Plot la matrice de confusion pour le training set
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx_tr, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Classes prédites')
plt.ylabel('Classes réelles')
plt.title('Matrice de confusion - Training Set')
plt.show()

# Tester le modèle sur le testing set
predictions_test = model.predict(X_test)
predictions_test = [np.argmax(x) for x in predictions_test]

accuracy_test = accuracy_score(np.argmax(y_test, axis=1), predictions_test)
print('Testing Accuracy = %.4f' % accuracy_test)

f1_test = f1_score(np.argmax(y_test, axis=1), predictions_test, average='weighted')
print('Testing F1 score = %.4f' % f1_test)

confusion_mtx_test = confusion_matrix(np.argmax(y_test, axis=1), predictions_test)

# Plot la matrice de confusion pour le testing set
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx_test, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Classes prédites')
plt.ylabel('Classes réelles')
plt.title('Matrice de confusion - Testing Set')
plt.show()

In [ ]:
# Classification Report
y_test_labels = [np.argmax(label) for label in y_test] # Récupérer les labels du testing set
# Afficher le classification report
print("Classification Report:\n", classification_report(y_test_labels, predictions_test, target_names=classes))